## Optional
Represents a value that may or may not be present. The main use case of `Optional` is when we are returning a value from a method and that value may or may not be present. It was added in Java 1.8.

To illustrate the need for `Optional`, we consider the next example. Consider a `search` method on a stream which returns `null` in case the search did not yield a result.

In [ ]:
// Below code returns NullPointerException if no red coloured fruit was found
fruits.stream()
    .search(fruit -> fruit.getColor().equals("RED"))
    .quantity();

To prevent NPE, we can do the following:

In [ ]:
Fruit fruit = fruits.stream()
    .search(fruit -> fruit.getColor().equals("RED"));

int quantity = fruit == null ? 0 : fruit.getQuantity();

The `Optional<T>` class was added in Java 8. An optional can be in two states:
- contains non null reference to `T`
- is empty

There are also equivalent primitives `OptionalInt`, `OptionalFloat`, `OptionalLong`, etc. In the previous example, where we were using the search method, Java provides an alternative `findFirst` which returns an `Optional`:

In [ ]:
// One difference to note as compared to search is that in this case the predicate
// is used upstream, and not in the findFirst method itself
Optional<Fruit> fruit = fruits.stream()
    .filter(fruit -> fruit.getColor().equals("RED"))
    .findFirst();

// If we had used int quantity = fruit.get().getQuantity(); we would have got
// NoSuchElementException when fruit optional was empty
int quantity = fruit.isPresent() ? fruit.get().getQuantity() : 0;

The above code uses two of the methods of `Optional` - `isPresent` which checks for empty and `get` which retreives the value associated with optional. However, the above case is no different than explicitly doing null checks like in example code in previous section.

Also, NEVER call `get` unless we're sure that the optional is non-empty.

The better alternative is to use the `orElse` methods:
- `orElse(<default_value>)` : return the value if present, else return the provided default value
- `orElseGet(<supplier_function>)` : return the value if present, else use the supplier to get default value
- `orElseThrow(<exception_object>)` : return the value if present, else throw exception provided

In [ ]:
int quantity = fruits.stream()
    .filter(fruit -> fruit.getColor().equals("RED"))
    .findFirst()
    .orElse(new Fruit())
    .getQuantity();

## Map and Filter
Are two available methods on an optional;
- `map` : if the optional is non-empty, it transforms the value into another in an optional. Else if the optional is empty it simply returns the optional.
- `filter`: if the optional is empty return that empty optional. If the optional is non-empty, apply the predicate. If the predicate passes, return the value in optional; else return empty optional.

In [ ]:
int quantity = fruits.stream()
    .filter(fruit -> fruit.getColor().equals("RED"))
    .findFirst()
    .map(fruit -> fruit.getQuantity())
    .orElse(0);

## Creating Optional
Never initialise an `Optional` reference to `null`. Instead use the `of` and `empty` methods:

In [ ]:
Optional<String> nonEmpty = Optional.of("Hello!");

Optional<String> empty = Optional.empty();
empty.get(); // NoSuchElementException

What if we have a normal reference and want to convert it into `Optional`? We can use the following method:

In [ ]:
Fruit nullRef = null;

Optional<Fruit> fruitOptional = Optional.ofNullable(nullRef);
fruitOptional.equals(Optional.empty());  // true

Fruit fruit = new Fruit();
Optional<Fruit> anotherFruitOptional = Optional.ofNullable(fruit);
anotherFruitOptional.equals(Optional.of(fruit));  // true

The `map` method alongwith `ofNullable` can help us avoid multiple levels of null checks:

In [ ]:
/* Suppose the class structure be
Product
    ProductData
        Supplier
            supplierCode

And we want to get:
String id = product.getProductData().getSuppliers().get(0).getSupplierCode();
*/

String id = null;
if(product != null && product.getProductData() != null && product.getProductData().getSuppliers() != null) {
    Supplier supplier = product.getProductData().getSuppliers().get(0);
    if(supplier != null) {
        id = supplier.getSupplierCode();
    }
}

// Simplified with Optional as:
String id_ = Optional.ofNullable(product)
                .map(Product::getProductData)
                .map(ProductData::getSuppliers)
                .map(s -> s.get(0))
                .map(Supplier::getSupplierCode)
                .orElse(null);